In [ ]:
import pandas as pd
from sklearn.feature_extraction.text import TfidfVectorizer 
from sklearn.metrics.pairwise import cosine_similarity
from fastapi import FastAPI
import uvicorn
from sklearn.metrics.pairwise import linear_kernel

In [ ]:
df = pd.read_csv('DataSet.csv')

In [ ]:
app = FastAPI()

In [ ]:
new_df = df.copy()

In [ ]:
new_df['Rating'].fillna(new_df['Rating'].mean(), inplace=True)
new_df['Review Count'].fillna(new_df['Review Count'].mean, inplace=True)
new_df['Prerequisites'].fillna("No Prerequisites Required",inplace=True)
new_df['Affiliates'].fillna("None",inplace=True)
new_df['Level'].fillna("beginner",inplace=True)

In [ ]:
new_df.isnull().sum()

In [ ]:
new_df.head()

In [ ]:
new_df_2 = new_df[['Title','Level','Prerequisites','Skills Covered']]
new_df_2.head()

In [ ]:
vectorizer = TfidfVectorizer()
df_vector = vectorizer.fit_transform(new_df_2)

In [ ]:
similarity = cosine_similarity(df_vector)

In [ ]:
def recommend(user_preferences, new_df_2):
  new_df_2['features'] = new_df_2['Title'] + ' ' +  new_df_2['Level'] + ' ' +  new_df_2['Prerequisites'] + ' ' + new_df_2['Skills Covered']
  vectorizer = TfidfVectorizer(stop_words='english')
  course_matrix = vectorizer.fit_transform(new_df_2['features'])
  cosine_similarities = linear_kernel(course_matrix, course_matrix)
  user_course_index = new_df_2[new_df_2['Title'] == user_preferences]
  similarity_scores = list(enumerate(cosine_similarities[user_course_index]))
  sorted_courses = sorted(similarity_scores, key=lambda x: x[1], reverse=True)
  top_n_recommendations = [(new_df_2.iloc[idx]['Title'], score) for idx, score in sorted_courses[1:]]
  return top_n_recommendations

print(recommend('HTML', new_df_2))  
